In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df=pd.read_csv('TV_show_data.csv')


In [3]:
df.head()

,Name,Summary,Average Runtime,End Date,Premiere Date,Genres,Type,Language,Network,Rating,Schedule (days),Schedule (time),Total Seasons,Total Episodes,Character Names,Person Names,Official Site
0,Breaking Bad,"Breaking Bad follows protagonist Walter White,...",60.0,2008-01-20,2019-10-11,"['Drama', 'Crime', 'Thriller']",Scripted,English,AMC,9.2,['Sunday'],22:00,5,62,"0-Walter White, 1-Jesse Pinkman, 2-Skyler Whit...","0-Bryan Cranston, 1-Aaron Paul, 2-Anna Gunn, 3...",http://www.amc.com/shows/breaking-bad
1,Firefly,"Five hundred years in the future, a renegade c...",64.0,2002-09-20,2002-12-20,"['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,FOX,9.0,['Friday'],20:00,1,14,"0-Captain Malcolm ""Mal"" Reynolds, 1-Zoe Alleyn...","0-Nathan Fillion, 1-Gina Torres, 2-Alan Tudyk,...",NaN
2,Avatar: The Last Airbender,Water. Earth. Fire. Air. Only the Avatar was t...,30.0,2005-02-21,2008-07-19,"['Action', 'Adventure', 'Fantasy']",Animation,English,Nickelodeon,8.9,['Saturday'],11:00,3,61,"0-Aang, 1-Katara, 2-Sokka, 3-Zuko, 4-Appa, 5-M...","0-Zach Tyler, 1-Mae Whitman, 2-Jack De Sena, 3...",https://www.nick.com/shows/avatar
3,Sherlock,Sherlock Holmes and Dr. John Watson's adventur...,90.0,2010-07-25,2017-01-15,"['Drama', 'Crime', 'Mystery']",Scripted,English,BBC One,8.9,['Sunday'],21:00,4,12,"0-Sherlock Holmes, 1-Dr. John Watson","0-Benedict Cumberbatch, 1-Martin Freeman",http://www.bbc.co.uk/programmes/b018ttws
4,Attack on Titan,"Known in Japan as Shingeki no Kyojin, many yea...",26.0,2013-04-07,2023-11-04,"['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,NHK,8.9,['Saturday'],00:10,4,89,"0-Mikasa Ackerman, 1-Armin Arlert, 2-Eren Jaeger","0-Yui Ishikawa, 1-Marina Inoue, 2-Yûki Kaji",http://shingeki.tv/


In [7]:
df['Summary'][0]

"Breaking Bad follows protagonist Walter White, a chemistry teacher who lives in New Mexico with his wife and teenage son who has cerebral palsy. White is diagnosed with Stage III cancer and given a prognosis of two years left to live. With a new sense of fearlessness based on his medical prognosis, and a desire to secure his family's financial security, White chooses to enter a dangerous world of drugs and crime and ascends to power in this world. The series explores how a fatal diagnosis such as White's releases a typical man from the daily concerns and constraints of normal society and follows his transformation from mild family man to a kingpin of the drug trade."

In [8]:
df['Type'].value_counts()


Type
Scripted       1051
Reality         245
Animation        94
Documentary      85
Talk Show        17
Variety          17
Game Show         9
Sports            7
News              5
Panel Show        3
Name: count, dtype: int64

In [9]:
def get_df_info(df):
    print("\n\033[1mShape of DataFrame:\033[0m ", df.shape)
    print("\n\033[1mColumns in DataFrame:\033[0m ", df.columns.to_list())
    print("\n\033[1mData types of columns:\033[0m\n", df.dtypes)
    
    print("\n\033[1mInformation about DataFrame:\033[0m")
    df.info()
    
    print("\n\033[1mNumber of unique values in each column:\033[0m")
    for col in df.columns:
        print(f"\033[1m{col}\033[0m: {df[col].nunique()}")
        
    print("\n\033[1mNumber of null values in each column:\033[0m\n", df.isnull().sum())
    
    print("\n\033[1mNumber of duplicate rows:\033[0m ", df.duplicated().sum())
    
    print("\n\033[1mDescriptive statistics of DataFrame:\033[0m\n", df.describe().transpose())

# Call the function
get_df_info(df)


Shape of DataFrame:  (1533, 17)

Columns in DataFrame:  ['Name', 'Summary', 'Average Runtime', 'End Date', 'Premiere Date', 'Genres', 'Type', 'Language', 'Network', 'Rating', 'Schedule (days)', 'Schedule (time)', 'Total Seasons', 'Total Episodes', 'Character Names', 'Person Names', 'Official Site']

Data types of columns:
 Name                object
Summary             object
Average Runtime    float64
End Date            object
Premiere Date       object
Genres              object
Type                object
Language            object
Network             object
Rating             float64
Schedule (days)     object
Schedule (time)     object
Total Seasons        int64
Total Episodes       int64
Character Names     object
Person Names        object
Official Site       object
dtype: object

Information about DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1533 entries, 0 to 1532
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------       

In [10]:
df['Network'].value_counts()


Network
ABC                    146
NBC                    135
CBS                     99
FOX                     80
BBC One                 68
                      ... 
Pay-Per-View             1
Family Channel           1
Smithsonian Channel      1
ABC Entertains           1
ESPN                     1
Name: count, Length: 139, dtype: int64

In [11]:
df=df[['Name','Summary','Genres','Type','Language','Person Names']]
df.head()

,Name,Summary,Genres,Type,Language,Person Names
0,Breaking Bad,"Breaking Bad follows protagonist Walter White,...","['Drama', 'Crime', 'Thriller']",Scripted,English,"0-Bryan Cranston, 1-Aaron Paul, 2-Anna Gunn, 3..."
1,Firefly,"Five hundred years in the future, a renegade c...","['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,"0-Nathan Fillion, 1-Gina Torres, 2-Alan Tudyk,..."
2,Avatar: The Last Airbender,Water. Earth. Fire. Air. Only the Avatar was t...,"['Action', 'Adventure', 'Fantasy']",Animation,English,"0-Zach Tyler, 1-Mae Whitman, 2-Jack De Sena, 3..."
3,Sherlock,Sherlock Holmes and Dr. John Watson's adventur...,"['Drama', 'Crime', 'Mystery']",Scripted,English,"0-Benedict Cumberbatch, 1-Martin Freeman"
4,Attack on Titan,"Known in Japan as Shingeki no Kyojin, many yea...","['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,"0-Yui Ishikawa, 1-Marina Inoue, 2-Yûki Kaji"


In [12]:
df['Person Names'][0]


'0-Bryan Cranston, 1-Aaron Paul, 2-Anna Gunn, 3-RJ Mitte, 4-Dean Norris, 5-Betsy Brandt, 6-Bob Odenkirk, 7-Steven Michael Quezada, 8-Jonathan Banks, 9-Giancarlo Esposito, 10-Charles Baker, 11-Matt L. Jones, 12-Christopher Cousins, 13-Jesse Plemons, 14-Lavell Crawford, 15-Michael Shamus Wiles, 16-Krysten Ritter, 17-Ray Campbell, 18-Carmen Serano, 19-Tina Parker, 20-Emily Rios, 21-Laura Fraser, 22-Mark Margolis, 23-Ian Posada, 24-Kevin Rankin, 25-David House, 26-Jeremiah Bitsui, 27-David Costabile, 28-Michael Bowen, 29-Todd Terry, 30-Daniel Moncada, 31-Jessica Hecht, 32-Luis Moncada, 33-Bill Burr, 34-Patrick Sane, 35-Tait Fletcher, 36-Matthew T. Metzler, 37-Rodney Rush'

In [16]:
def extract_names(row):
    if isinstance(row, str):
        # Split the row by comma to separate each "index-name" pair
        pairs = row.split(', ')
        # Extract the names by splitting each pair by the hyphen and taking the second part
        names = [pair.split('-')[1] for pair in pairs if '-' in pair]
        return names
    else:
        return []  # Return an empty list if the row is not a string

# Apply the function to each row of the 'Person Names' column
df['Extracted Names'] = df['Person Names'].apply(extract_names)


In [17]:
df['Extracted Names']

0       [Bryan Cranston, Aaron Paul, Anna Gunn, RJ Mit...
1       [Nathan Fillion, Gina Torres, Alan Tudyk, More...
2       [Zach Tyler, Mae Whitman, Jack De Sena, Dante ...
3                  [Benedict Cumberbatch, Martin Freeman]
4                 [Yui Ishikawa, Marina Inoue, Yûki Kaji]
                              ...                        
1528    [Ted Allen, Alex Guarnaschelli, Curtis Stone, ...
1529                                                   []
1530    [Ester Steinberg, Scout Durwood, Jess Varley, ...
1531                             [Cedric the Entertainer]
1532                      [Pauline Collins, Denis Lawson]
Name: Extracted Names, Length: 1533, dtype: object

In [18]:
df = df.drop(columns=['Person Names'])
df.head()

,Name,Summary,Genres,Type,Language,Extracted Names
0,Breaking Bad,"Breaking Bad follows protagonist Walter White,...","['Drama', 'Crime', 'Thriller']",Scripted,English,"[Bryan Cranston, Aaron Paul, Anna Gunn, RJ Mit..."
1,Firefly,"Five hundred years in the future, a renegade c...","['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,"[Nathan Fillion, Gina Torres, Alan Tudyk, More..."
2,Avatar: The Last Airbender,Water. Earth. Fire. Air. Only the Avatar was t...,"['Action', 'Adventure', 'Fantasy']",Animation,English,"[Zach Tyler, Mae Whitman, Jack De Sena, Dante ..."
3,Sherlock,Sherlock Holmes and Dr. John Watson's adventur...,"['Drama', 'Crime', 'Mystery']",Scripted,English,"[Benedict Cumberbatch, Martin Freeman]"
4,Attack on Titan,"Known in Japan as Shingeki no Kyojin, many yea...","['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,"[Yui Ishikawa, Marina Inoue, Yûki Kaji]"


In [19]:
df['Summary']=df['Summary'].apply(lambda x:x.split())


In [20]:
df.head()

,Name,Summary,Genres,Type,Language,Extracted Names
0,Breaking Bad,"[Breaking, Bad, follows, protagonist, Walter, ...","['Drama', 'Crime', 'Thriller']",Scripted,English,"[Bryan Cranston, Aaron Paul, Anna Gunn, RJ Mit..."
1,Firefly,"[Five, hundred, years, in, the, future,, a, re...","['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,"[Nathan Fillion, Gina Torres, Alan Tudyk, More..."
2,Avatar: The Last Airbender,"[Water., Earth., Fire., Air., Only, the, Avata...","['Action', 'Adventure', 'Fantasy']",Animation,English,"[Zach Tyler, Mae Whitman, Jack De Sena, Dante ..."
3,Sherlock,"[Sherlock, Holmes, and, Dr., John, Watson's, a...","['Drama', 'Crime', 'Mystery']",Scripted,English,"[Benedict Cumberbatch, Martin Freeman]"
4,Attack on Titan,"[Known, in, Japan, as, Shingeki, no, Kyojin,, ...","['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,"[Yui Ishikawa, Marina Inoue, Yûki Kaji]"


In [21]:
df['Extracted Names']=df['Extracted Names'].apply(lambda x:[i.replace(" ","")for i in x])


In [22]:
df.head()

,Name,Summary,Genres,Type,Language,Extracted Names
0,Breaking Bad,"[Breaking, Bad, follows, protagonist, Walter, ...","['Drama', 'Crime', 'Thriller']",Scripted,English,"[BryanCranston, AaronPaul, AnnaGunn, RJMitte, ..."
1,Firefly,"[Five, hundred, years, in, the, future,, a, re...","['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,"[NathanFillion, GinaTorres, AlanTudyk, MorenaB..."
2,Avatar: The Last Airbender,"[Water., Earth., Fire., Air., Only, the, Avata...","['Action', 'Adventure', 'Fantasy']",Animation,English,"[ZachTyler, MaeWhitman, JackDeSena, DanteBasco..."
3,Sherlock,"[Sherlock, Holmes, and, Dr., John, Watson's, a...","['Drama', 'Crime', 'Mystery']",Scripted,English,"[BenedictCumberbatch, MartinFreeman]"
4,Attack on Titan,"[Known, in, Japan, as, Shingeki, no, Kyojin,, ...","['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,"[YuiIshikawa, MarinaInoue, YûkiKaji]"


In [23]:
df['Extracted Names'] = df['Extracted Names'].apply(lambda x: x[:3])
df.head()

,Name,Summary,Genres,Type,Language,Extracted Names
0,Breaking Bad,"[Breaking, Bad, follows, protagonist, Walter, ...","['Drama', 'Crime', 'Thriller']",Scripted,English,"[BryanCranston, AaronPaul, AnnaGunn]"
1,Firefly,"[Five, hundred, years, in, the, future,, a, re...","['Drama', 'Adventure', 'Science-Fiction']",Scripted,English,"[NathanFillion, GinaTorres, AlanTudyk]"
2,Avatar: The Last Airbender,"[Water., Earth., Fire., Air., Only, the, Avata...","['Action', 'Adventure', 'Fantasy']",Animation,English,"[ZachTyler, MaeWhitman, JackDeSena]"
3,Sherlock,"[Sherlock, Holmes, and, Dr., John, Watson's, a...","['Drama', 'Crime', 'Mystery']",Scripted,English,"[BenedictCumberbatch, MartinFreeman]"
4,Attack on Titan,"[Known, in, Japan, as, Shingeki, no, Kyojin,, ...","['Action', 'Adventure', 'Anime', 'Fantasy']",Animation,Japanese,"[YuiIshikawa, MarinaInoue, YûkiKaji]"


In [24]:
df['Genres'] = df['Genres'].apply(lambda x: [x])


In [25]:
df['tags']=df['Summary']+df['Genres']+df['Extracted Names']


In [26]:
df.head()


,Name,Summary,Genres,Type,Language,Extracted Names,tags
0,Breaking Bad,"[Breaking, Bad, follows, protagonist, Walter, ...","[['Drama', 'Crime', 'Thriller']]",Scripted,English,"[BryanCranston, AaronPaul, AnnaGunn]","[Breaking, Bad, follows, protagonist, Walter, ..."
1,Firefly,"[Five, hundred, years, in, the, future,, a, re...","[['Drama', 'Adventure', 'Science-Fiction']]",Scripted,English,"[NathanFillion, GinaTorres, AlanTudyk]","[Five, hundred, years, in, the, future,, a, re..."
2,Avatar: The Last Airbender,"[Water., Earth., Fire., Air., Only, the, Avata...","[['Action', 'Adventure', 'Fantasy']]",Animation,English,"[ZachTyler, MaeWhitman, JackDeSena]","[Water., Earth., Fire., Air., Only, the, Avata..."
3,Sherlock,"[Sherlock, Holmes, and, Dr., John, Watson's, a...","[['Drama', 'Crime', 'Mystery']]",Scripted,English,"[BenedictCumberbatch, MartinFreeman]","[Sherlock, Holmes, and, Dr., John, Watson's, a..."
4,Attack on Titan,"[Known, in, Japan, as, Shingeki, no, Kyojin,, ...","[['Action', 'Adventure', 'Anime', 'Fantasy']]",Animation,Japanese,"[YuiIshikawa, MarinaInoue, YûkiKaji]","[Known, in, Japan, as, Shingeki, no, Kyojin,, ..."


In [27]:
new_df=df[['Name','tags']]


In [28]:
new_df.head()


,Name,tags
0,Breaking Bad,"[Breaking, Bad, follows, protagonist, Walter, ..."
1,Firefly,"[Five, hundred, years, in, the, future,, a, re..."
2,Avatar: The Last Airbender,"[Water., Earth., Fire., Air., Only, the, Avata..."
3,Sherlock,"[Sherlock, Holmes, and, Dr., John, Watson's, a..."
4,Attack on Titan,"[Known, in, Japan, as, Shingeki, no, Kyojin,, ..."


In [29]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


/tmp/ipykernel_847/2363790582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [30]:
new_df.head()

,Name,tags
0,Breaking Bad,"Breaking Bad follows protagonist Walter White,..."
1,Firefly,"Five hundred years in the future, a renegade c..."
2,Avatar: The Last Airbender,Water. Earth. Fire. Air. Only the Avatar was t...
3,Sherlock,Sherlock Holmes and Dr. John Watson's adventur...
4,Attack on Titan,"Known in Japan as Shingeki no Kyojin, many yea..."


In [31]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


/tmp/ipykernel_847/3329932311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [32]:
ps=PorterStemmer()

In [33]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)  

In [34]:
new_df['tags']=new_df['tags'].apply(stem)


/tmp/ipykernel_847/3293759246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [35]:
cv=CountVectorizer(max_features=5000,stop_words='english')


In [36]:
vectors=cv.fit_transform(new_df['tags']).toarray()


In [37]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
 cv.get_feature_names_out()


array(['000', '10', '100', ..., 'zombi', 'zone', 'zoo'], dtype=object)

In [39]:
similarity=cosine_similarity(vectors)


In [40]:
similarity[0]

array([1.        , 0.04012862, 0.09128709, ..., 0.08421519, 0.09622504,
       0.0949158 ])

In [41]:
def recommend(movie):
    movie_index=new_df[new_df['Name']==movie].index[0]
    distances=similarity[movie_index]
    movie_list= sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print (new_df.iloc[i[0]].Name)

In [42]:
recommend('Attack on Titan')


The Legend of Korra
Durarara!!
Beauty & the Beast
Dominion
Nikita


In [43]:
recommend('One Piece')

Dungeons & Dragons
Young Justice
Celebrity Big Brother
Kill la Kill
Sonic Boom
